In [1]:
import torch
import numpy as np
from torchvision import datasets
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
import matplotlib.pyplot as plt
from torch.utils.data.sampler import SubsetRandomSampler

In [ ]:
class UNet(nn.Module,in_channels,out_channels):
    def __init__(self):
        super(UNet,self).__init__()
        
        self.in_channels = in_channels
        self.out_channels = out_channels
        
        #UNet Arch:
        #1)Encoder
        self.conv1  = nn.Conv2d(in_channels,32,7,1,3)
        self.conv2  = nn.Conv2d(32,32,7,1,3)
        
        self.conv3  = nn.Conv2d(32,64,5,1,2)
        self.conv4  = nn.Conv2d(64,64,5,1,2)
        
        self.conv5  = nn.Conv2d(64,128,3,1,1)
        self.conv6  = nn.Conv2d(128,128,3,1,1)
        
        self.conv7  = nn.Conv2d(128,256,3,1,1)
        self.conv8  = nn.Conv2d(256,256,3,1,1)
        
        self.conv9  = nn.Conv2d(256,512,3,1,1)
        self.conv10 = nn.Conv2d(512,512,3,1,1)
        
        #1)Decoder
        self.up_sample = nn.up_sample(scale_factor = 2, mode='bilinear')
        
        self.conv11 = nn.conv2(512,512,3,1,1)
        self.conv12 = nn.conv2(512,512,3,1,1)
        
        self.conv13 = nn.conv2(512,256,3,1,1)
        self.conv14 = nn.conv2(256,256,3,1,1)
        
        self.conv15 = nn.conv2(256,128,3,1,1)
        self.conv16 = nn.conv2(128,128,3,1,1)
        
        self.conv17 = nn.conv2(128,64,3,1,1)
        self.conv18 = nn.conv2(64,64,3,1,1)
        
        self.conv19 = nn.conv2(64,32,3,1,1)
        self.conv20 = nn.conv2(32,out_channels,3,1,1)